In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

     ------------------------------------- 816.1/816.1 kB 50.5 kB/s eta 0:00:00
     ------------------------------------- 394.8/394.8 kB 33.5 kB/s eta 0:00:00
     ------------------------------------- 241.3/241.3 kB 45.2 kB/s eta 0:00:00
     ------------------------------------- 366.0/366.0 kB 40.4 kB/s eta 0:00:00
     --------------------------------------- 61.0/61.0 kB 95.6 kB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 42.7 kB/s eta 0:00:00
     --------------------------------------- 50.7/50.7 kB 46.2 kB/s eta 0:00:00
     --------------------------------------- 76.9/76.9 kB 68.9 kB/s eta 0:00:00
     --------------------------------------- 49.4/49.4 kB 67.8 kB/s eta 0:00:00
     --------------------------------------- 53.0/53.0 kB 85.5 kB/s eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 32.9 kB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing

     --------------------------------------- 10.8/10.8 MB 77.1 kB/s eta 0:00:00
     ------------------------------------- 798.7/798.7 kB 97.6 kB/s eta 0:00:00


In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [3]:
# Get your API keys from openai, you will need to create an account. 
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
import os
os.environ["OPENAI_API_KEY"] = "..."

In [4]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

ModuleNotFoundError: No module named 'google'

In [ ]:
# location of the pdf file/files. 
reader = PdfReader('/content/gdrive/My Drive/data/2023_GPT4All_Technical_Report.pdf')

In [ ]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
raw_text[:100]

In [ ]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

In [ ]:
texts[0]

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [ ]:
docsearch = FAISS.from_texts(texts, embeddings)

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [ ]:
query = "who are the authors of the article?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)